# Functions III: more on function arguments

## Handling mutable function arguments
Let us now explore the local namespaces of functions which have input arguments.

In [ ]:
x = 3
y = 6

def g(arg1,arg2):
    print(arg1 is x, arg2 is y)
    print(locals())
    return(arg1*arg2)

As the above example shows, the arguments of a function are essentially local names bound to the memory locations which the respective variables in the main program are bound to. This observation is important to keep in mind, particularly in the context of providing functions with mutable objects. Consider the following function, designed to return the result of entrywise doubling and then summing a list:

In [ ]:
x = [1,2,3]

def double_and_sum(arg):
    for i in range(len(arg)):
        arg[i] = 2*arg[i]
    return(sum(arg))

double_and_sum(x), x

As this example highlights, an unintended outcome of calling `double_and_sum` is that the global variable `x`  has also been changed. The reasons for this are the same as given in Tutorial 2: line 5 in the above is a mutation which changes the data at the memory location which both the global variable `x` and the local variable `arg` are bound to. Once one is aware of this risk then often it is not hard to avoid, as the following alternative implementations demonstrate.

In [ ]:
# The expensive but safe way, copy the list first then run the same code
def copy_double_and_sum(arg):
    new_list = arg.copy() # copy the list and process that instead, expensive computationally!
    for i in range(len(new_list)):
        new_list[i] = 2*new_list[i]
    return(sum(new_list))

# A simpler way...
def double_and_sum_simple(arg):
    return sum(2*arg) # achieve the same result without performing a mutation

In [ ]:
x = [1,2,3]
print(copy_double_and_sum(x), x)
print(double_and_sum_simple(x), x)

Functions which do not change the value of any input variables, or variables outside of their scope, are typically easier to test and safer to use and combine. For these reasons, if you can, its best to implement functions in this way. For example, suppose we want to regularly update lists by appending their average value.

In [ ]:
# First approach, which is potentially more error prone if your code gets long and complicated
x = [2,2,3,4,5]

def append_average(list_arg):
    list_arg.append(sum(list_arg)/len(list_arg))
    
append_average(x)
x

In [ ]:
# An approach which is arguably safer...
x = [2,2,3,4,5]

def list_average(list_arg):
    return sum(list_arg)/len(list_arg)

x.append(list_average(x)) # mutation happens back in the main program
x

## Handling an unknown number of input arguments
In some cases, we might not know how many arguments a function needs to accept. For example, consider the following function:

In [ ]:
def add(a, b):
    return(a+b)
add(2, 2)

This works, but only for two numbers. 

In [ ]:
add(2, 2, 2)
# ---

Of course, we could define a version of "add" that works with three numbers, or four....but there's a better way. 

The special `*args` argument can be passed to the function. Within the function scope, `args` (no `*` asterisk) is then a tuple of all **positional** arguments passed to the function. So, we can write a general `add` this way:

In [16]:
def better_add(*args):
    total = 0
    # args is a tuple containing all of the inputs
    for a in args:
        total += a
    return(total)

better_add(2, 2), better_add(2, 2, 2), better_add(2, 2, 2, 2, 2)

(4, 6, 10)

To be clear, using the name `args` is just a convention, one could equally use another name like `numbers` for instance. What matters in this context is the $*$ unpacking operator. This operator can be applied to any iterable and returns the contents of that iterable. For example, the following no longer print the iterables in question but their contents.

In [12]:
s = "hello"
print(*s)
my_list = [1,2,3,4]
print(*my_list)
my_tuple = (1,2,3,4)
print(*my_tuple)

h e l l o
1 2 3 4
1 2 3 4


Therefore, when we provide a function with a `*args` argument a sequence of unknown positional arguments, the function interprets these positional arguments as an unpacked tuple, called `args`. Therefore, the variable `args` inside the function is a tuple of these unknown positional inputs. Note that by automatically setting the type of `args` as a tuple Python is stopping us from accidently mutating the data of these unknown positional arguments.

In some cases, you might not be sure how many **keyword** arguments your function needs to accept. In this case, use `**kwargs` (with two `*` asterisks). Having done so, `kwargs` will be available as a dictionary within the function scope. 

In [28]:
def favorites(**kwargs):    
    for key, val in kwargs.items():
        print("My favorite " + key + " is " + val + ".")

favorites(cuisine = "Greek", music = "Jazz", sport = 'climbing')
# ---

My favorite cuisine is Greek.
My favorite music is Jazz.
My favorite sport is climbing.


Again the name `kwargs` is not important, we could for instance use `pairs`, what matters is the `**` unpacking operator. Analagous to the `*` operator for iterables, `**` unpacks dictionaries. When we provide a function with a `**kwargs` argument a sequence of unknown keyword arguments, the function interprets these keyword arguments as an unpacked dictionary called `kwargs`. Therefore, the variable `kwargs` inside the function is a dictionary containing the unknown keyword arguments.

Note that it is possible to use `*args` and `**kwargs` together. Since positional arguments always come first, it's necessary to use `*args` before `**kwargs`. 

## Anonymous functions
$\lambda$-expressions provide a concise way to create very simple functions. While the syntax is somewhat odd, it's quite readable once you know the correct idiom. 

In [ ]:
# "double is the function that takes x and multiplies it by 2"
double = lambda x: 2*x 

# same as:
# def double(x):
#     return 2*x

double(4)

In [ ]:
# "second_char is the function that returns the second character of a string s"
second_char = lambda s: s[1] 

In [ ]:
second_char("PIC16A")

$\lambda$-expressions are extremely useful when a relatively simple function is required, for example when sorting lists. 

In [29]:
# sort a list into even and odd:

L = [4, 6, 9, 3, 4, 6, 7, 3, 2, 0, 9, 5]
L.sort(key = lambda x: (x % 2) == 1)
L

[4, 6, 4, 6, 2, 0, 9, 3, 7, 3, 9, 5]

In [33]:
# decreasing order within even and odd groups
L.sort(key = lambda x: ((x % 2) == 1, -x))
L

[6, 6, 4, 4, 2, 0, 9, 9, 7, 5, 3, 3]

In [31]:
# lambda functions also accept multiple arguments
multiply = lambda x,y: x*y
multiply(2, 3)

6

$\lambda$-expressions which are complicated are hard to read and debug. Generally speaking, these expressions should not contain control flow statements, and should not be longer than a single, 80-character line of code. If your $\lambda$-expression is getting complex, use an explicitly-defined function instead. 